In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Helper functions


1. rename_column()

> Replaces spaces with underscore


2. to_numeric()

> Does its own magic

3. to_datetimes()

> Can explain what do the function parameters do. You can also give some examples of the dataset, after passing into this function, what is the output. Make it look like that I only need to read this documentation to know everything what the function is doing along with its implementation.

4. compute_efw_centile()

> what centile are we using? use what formula to compute? something more context if i want to be techincal.


remember, putting enough info in here for someone to know the high level. and if they are interested, they can dwelve into the code to see. no need to do guesswork from the code what it's doing.




In [ ]:
# convert columns to lower case, replace spaces with underscore, add alternative names if needed
def rename_column(df, alternative = None):
  df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_', regex=False).str.replace(r'[, _]+', '_')
  if alternative != None:
    df = df.rename(columns = alternative)
  return df

def to_numeric(df, col):
  len_before = len(df[df[col].notna()])
  df[col] = df[col].applymap(pd.to_numeric, errors='coerce')
  len_after = len(df[df[col].notna()])
  return df

# convert all formats of date and time from string formst to datetime format
# Input: "10/11/12 09:24:01"
# Output: Timestamp(2012-11-10 09:24:01)
def to_datetimes(df, cols, day_first = False, time = False):
  for col in cols:
    if day_first:
      df[col] = pd.to_datetime(df[col], format = '%Y-%m-%d', errors = 'coerce')
    else:
      if time:
        df[col] = pd.to_datetime(df[col], format = '%Y.%m.%d %H:%M:%s', errors = 'coerce').fillna(
          pd.to_datetime(df[col], format = '%Y//%m/%d %H:%M:%s', errors = 'coerce')).fillna(
          pd.to_datetime(df[col], format = '%Y-%m-%d %H:%M:%s', errors = 'coerce')).fillna(
          pd.to_datetime(df[col], format = '%Y/%m/%d %H:%M:%s', errors = 'coerce')).fillna(
          pd.to_datetime(df[col], format = '%Y.%m%d %H:%M:%s', errors = 'coerce'))
      else:
        df[col] = pd.to_datetime(df[col], format = '%m.%d.%Y', errors = 'coerce').fillna(
          pd.to_datetime(df[col], format = '%m//%d/%Y', errors = 'coerce')).fillna(
          pd.to_datetime(df[col], format = '%d-%b-%Y', errors = 'coerce')).fillna(
          pd.to_datetime(df[col], format = '%m/%d/%Y', errors = 'coerce')).fillna(
          pd.to_datetime(df[col], format = '%m.%d%Y', errors = 'coerce'))
  return df

# Benchmark source: https://srhr.org/fetalgrowthcalculator/#/
# Formula: https://www.cuemath.com/linear-interpolation-formula/
def compute_efw_centile(df, centile_df):
  centile_df = centile_df.rename(columns={'GA' : 'ga'})

  df = pd.merge(df, centile_df, on = 'ga', how = 'left')
  refCentile = list(centile_df.columns)[1:]
  ranges = df[[2.5, 5, 10, 25, 50, 75, 90, 95, 97.5]].values.tolist()
  efw = df['efw'].values.tolist()
  centile = []

  for i in range(len(efw)):
    if efw[i] >= ranges[i][0] and efw[i] <= ranges[i][-1]:
      for j in range(len(ranges[i])):
        if efw[i] == ranges[i][j]:
          centile.append(refCentile[j])
          break
        if efw[i] < ranges[i][j]:
          centile.append(refCentile[j] - (refCentile[j] - refCentile[j - 1]) * (ranges[i][j] - efw[i]) / (ranges[i][j] - ranges[i][j - 1]))
          break
    else:
      centile.append(0)

  df['efw_centile'] = centile
  df.drop(df[df['efw_centile'] == 0].index, inplace = True)
  df['cur_sga'] = (df['efw_centile'] <= 10).astype(int)
  return df

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/SGA/S dataset/data/IUGR Data Collection Sheet V2_05042021 (Prospective Data)_Updated05012023(No identifiers) (1).xlsx', skiprows = 5)
df = df.iloc[:, :-9]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 32 columns):
 #   Column                                                     Non-Null Count  Dtype         
---  ------                                                     --------------  -----         
 0   Subject Screening Number                                   196 non-null    object        
 1   Date of Consent                                            196 non-null    datetime64[ns]
 2   Date of Subject Screened                                   196 non-null    datetime64[ns]
 3   Subject Eligibility                                        196 non-null    object        
 4   Enrolled to Study                                          196 non-null    object        
 5   Subject Enrollment Number                                  196 non-null    object        
 6   Mother's Age at Delivery                                   196 non-null    int64         
 7   Ethnics (Chinese = 1, Malay = 2, In

In [ ]:
df = rename_column(df, alternative = {'mother\'s_age_at_delivery' : 'm_age', 'biparietal_diameter_(mm)' : 'bpd', 'head_circumference_(mm)' : 'hc', 'cisterna_magna_(mm)' : 'cm', 'estimated_fetal_weight_(g)' : 'efw', 'nuchal_fold_thickness_(mm)' : 'nf', 'femur_length_(mm)' : 'fl', 'abdominal_circumference_(mm)' : 'ac', 'birthweight_(g)' : 'bw', 'baby_gender_(f/m)' : 'gender'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 32 columns):
 #   Column                                                 Non-Null Count  Dtype         
---  ------                                                 --------------  -----         
 0   subject_screening_number                               196 non-null    object        
 1   date_of_consent                                        196 non-null    datetime64[ns]
 2   date_of_subject_screened                               196 non-null    datetime64[ns]
 3   subject_eligibility                                    196 non-null    object        
 4   enrolled_to_study                                      196 non-null    object        
 5   subject_enrollment_number                              196 non-null    object        
 6   m_age                                                  196 non-null    int64         
 7   ethnics_(chinese_=_1_malay_=_2_indian_=_3_others_=_4)  196 non-null    

<ipython-input-20-93157c71cc70>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_', regex=False).str.replace(r'[, _]+', '_')


In [ ]:
# ease of caclculation, ig
df['ga'] = df['gestational_age_scanned_(week)'] * 7 + df['gestational_age_scanned_(day)']
df['birth_ga'] = df['gestational_week'] * 7 + df['gestational_day']
# diabetes mellitus(DM) / gestational diabetes mellitus(GDM)
df['diabetes'] = df['antenatal_medical_conditions'] == 'DM/GDM'

In [ ]:
# convert mm to cm for further calculation
hc = df['hc'] / 10 # maybe can rename to hc_in_cm
ac = df['ac'] / 10 # maybe can rename to ac_in_cm
fl = df['fl'] / 10 # maybe can rename to fl_in_cm

# Hadlock Formula: https://perinatology.com/calculators/Estimation%20of%20Fetal%20Weight%20and%20Age.htm
df['bpd'] = np.where(df['bpd'].isna(), 10 * (np.log10(df['efw']) - 1.3596 - 0.0064 * hc - 0.0424 * ac - 0.174 * fl + 0.00386 * ac * fl) / (0.00061 * ac), df['bpd'])
df['efw'] = np.where(df['efw'].isna(), round(10 ** (1.3596 + 0.0064 * hc + 0.0424 * ac + 0.174 * fl + 0.00061 * df['bpd'] / 10 * ac - 0.00386 * ac * fl)), df['efw'])

In [ ]:
df['bw'] = pd.to_numeric(df['bw'], errors='coerce')
df = df.dropna(subset=['bw'])
df.drop(df[df['gender'].isna()].index, axis = 0, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 195
Data columns (total 35 columns):
 #   Column                                                 Non-Null Count  Dtype         
---  ------                                                 --------------  -----         
 0   subject_screening_number                               188 non-null    object        
 1   date_of_consent                                        188 non-null    datetime64[ns]
 2   date_of_subject_screened                               188 non-null    datetime64[ns]
 3   subject_eligibility                                    188 non-null    object        
 4   enrolled_to_study                                      188 non-null    object        
 5   subject_enrollment_number                              188 non-null    object        
 6   m_age                                                  188 non-null    int64         
 7   ethnics_(chinese_=_1_malay_=_2_indian_=_3_others_=_4)  188 non-null    

<ipython-input-28-c3d69aa6fa1d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['gender'].isna()].index, axis = 0, inplace = True)


In [ ]:
centile_df = pd.read_excel('/content/drive/MyDrive/SGA/Prev/EFW centile.xlsx')
df = compute_efw_centile(df, centile_df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 187
Data columns (total 46 columns):
 #   Column                                                 Non-Null Count  Dtype         
---  ------                                                 --------------  -----         
 0   subject_screening_number                               187 non-null    object        
 1   date_of_consent                                        187 non-null    datetime64[ns]
 2   date_of_subject_screened                               187 non-null    datetime64[ns]
 3   subject_eligibility                                    187 non-null    object        
 4   enrolled_to_study                                      187 non-null    object        
 5   subject_enrollment_number                              187 non-null    object        
 6   m_age                                                  187 non-null    int64         
 7   ethnics_(chinese_=_1_malay_=_2_indian_=_3_others_=_4)  187 non-null    

## I21 BW chart data

* Can explain what are we doing here.
* Explain how do you generate the Ground Truth (GT). Why 2.5? what does 2.5 mean? kg? g? centile? lbs? if specify here in the documentation, would be better.





In [ ]:
gt = pd.read_csv('/content/drive/MyDrive/SGA/Ref_Centile/I21_BW.csv')
gt.columns = gt.columns.str.lower()
gt.rename(columns = {'ga' : 'birth_ga'}, inplace = True)
df = to_numeric(df, ['bw'])
df = pd.merge(df, gt, on = ['birth_ga', 'gender'], how = 'left')
df['sga'] = df['bw'] <= df['p_10'] # sga is bw less than 10th centile
df['lbw'] = df['bw'] <= 2.5 # centile
df['sc'] = df['cur_sga'] ^ df['sga']
df = to_numeric(df, ['sga', 'lbw', 'sc'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Data columns (total 56 columns):
 #   Column                                                 Non-Null Count  Dtype         
---  ------                                                 --------------  -----         
 0   subject_screening_number                               187 non-null    object        
 1   date_of_consent                                        187 non-null    datetime64[ns]
 2   date_of_subject_screened                               187 non-null    datetime64[ns]
 3   subject_eligibility                                    187 non-null    object        
 4   enrolled_to_study                                      187 non-null    object        
 5   subject_enrollment_number                              187 non-null    object        
 6   m_age                                                  187 non-null    int64         
 7   ethnics_(chinese_=_1_malay_=_2_indian_=_3_others_=_4)  187 non-null    

In [ ]:
df['gender'].replace({'F' : 0, 'M' : 1}, inplace = True)
df = to_numeric(df, ['gender'])
# replace 13/011/2022 to 13/11/2022, fix invalid delivery date
df['delivery_date'] = np.where(df['delivery_date'] == '13/011/2022', '13/11/2022', df['delivery_date'])
df['delivery_date'] = pd.to_datetime(df['delivery_date'])

<ipython-input-31-86047d5cfc91>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['delivery_date'] = pd.to_datetime(df['delivery_date'])


In [ ]:
# Remove unneccessary / irrelevant / lack-of-data columns
df = df[['subject_screening_number', 'subject_enrollment_number', 'm_age', 'bpd', 'hc', 'cerebellum_transverse_(mm)', 'cm', 'efw', 'nf', 'ac', 'fl', 'anterior_horn_of_lateral_v_(mm)', 'posterior_horn_of_lateral_v_(mm)', 'hemisphere_(mm)', 'uterine_ri', 'uterine_pi', 'bw', 'gender', 'delivery_date', 'ga', 'birth_ga', 'diabetes', 'efw_centile', 'cur_sga', 'sga', 'lbw', 'sc']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   subject_screening_number          187 non-null    object        
 1   subject_enrollment_number         187 non-null    object        
 2   m_age                             187 non-null    int64         
 3   bpd                               187 non-null    float64       
 4   hc                                187 non-null    float64       
 5   cerebellum_transverse_(mm)        183 non-null    float64       
 6   cm                                182 non-null    float64       
 7   efw                               187 non-null    float64       
 8   nf                                182 non-null    float64       
 9   ac                                187 non-null    float64       
 10  fl                                187 non-null    